### Imports

In [86]:
import pandas as pd
import sqlite3
import os

### Connect to the DB

In [87]:
db_file = "sqlite3_group10.db"

# Delete db file if it exists
if os.path.exists(db_file):
    os.remove(db_file)

conn = sqlite3.connect(db_file)
cur = conn.cursor()

### Read CSV files

In [88]:
csv_folder = "archive"

completed_highschool_df = pd.read_csv(f"{csv_folder}/PercentOver25CompletedHighSchool.csv", encoding="ISO-8859-1")
police_killings_df = pd.read_csv(f"{csv_folder}/PoliceKillingsUS.csv", encoding="ISO-8859-1")
race_by_city_df = pd.read_csv(f"{csv_folder}/ShareRaceByCity.csv", encoding="ISO-8859-1")

### Create Tables

##### Create State Table

In [89]:
create_state_query = '''
CREATE TABLE state (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  acronym CHAR(2) NOT NULL
);
'''

cur.execute(create_state_query)

##### Create City Table

In [90]:
create_city_query = '''
CREATE TABLE city (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  state_id INTEGER NOT NULL,
  FOREIGN KEY (state_id) REFERENCES state(id),
  UNIQUE(id, state_id)
);
'''

cur.execute(create_city_query)

#### Create Percent_over_25_completed_highschool Table

In [91]:
create_completed_highschool_query = '''
CREATE TABLE percent_over_25_completed_highschool (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  percentage FLOAT NOT NULL,
  city_id INTEGER NOT NULL,
  FOREIGN KEY (city_id) REFERENCES city(id)
);
'''

cur.execute(create_completed_highschool_query)

#### Create Race Table

In [92]:
create_race_query = '''
CREATE TABLE race (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL
);
'''

cur.execute(create_race_query)

##### Create Share_race_by_city Table

In [93]:
create_share_race_by_city_query = '''
CREATE TABLE share_race_by_city (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  share FLOAT NOT NULL,
  race_id INTEGER NOT NULL,
  city_id INTEGER NOT NULL,
  FOREIGN KEY (city_id) REFERENCES city(id),
  FOREIGN KEY (race_id) REFERENCES race(id)
);
'''

cur.execute(create_share_race_by_city_query)

##### Create Police_killings Table

In [94]:
create_police_killings_query = '''
CREATE TABLE police_killings (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  date DATE NOT NULL,
  manner_of_death TEXT NOT NULL,
  armed TEXT,
  age INTEGER,
  gender TEXT NOT NULL,
  race_id INTEGER,
  city_id INTEGER,
  signs_of_mental_illness INTEGER NOT NULL,
  threat_level TEXT NOT NULL,
  flee TEXT,
  body_camera INTEGER NOT NULL,
  FOREIGN KEY (race_id) REFERENCES race(id),
  FOREIGN KEY (city_id) REFERENCES city(id)
);
'''

cur.execute(create_police_killings_query)

### Insert Data

---
#### State

Insert Data Into State Table

In [95]:
# Get all unique states
states_df = completed_highschool_df["Geographic Area"].drop_duplicates().to_frame()
# Rename columns to match table colum names
states_df.rename(columns={'Geographic Area': 'acronym'}, inplace=True)

states_df.to_sql("state", conn, if_exists='append', index = False)

51

Create Dict that maps a state to its id

In [96]:
cur.execute("SELECT * FROM state;")

rows = cur.fetchall();
state_to_id = {}
for row in rows:
  state_to_id[row[1]] = row[0]

---
#### City

Insert Data Into City Table

In [97]:
# Replaces redundent words with empty string
def remove_redundency(dataframe):
    dataframe['City'] = dataframe['City'].str.replace(' town', '')
    dataframe['City'] = dataframe['City'].str.replace(' city', '')
    dataframe['City'] = dataframe['City'].str.replace(' CDP', '')

In [98]:
# Remove city, town or CDP from city name (so that police killings cities match)
remove_redundency(completed_highschool_df)

# Get cities and their states
cities = completed_highschool_df.loc[:, ["City", "Geographic Area"]]

# Rename columns to match table colum names
cities.rename(columns={'City': 'name', "Geographic Area": "state_id"}, inplace=True)

# Convert states to state ids
cities["state_id"] = cities["state_id"].map(state_to_id)

cities.to_sql("city", conn, if_exists='append', index = False)

29329

Create Dict that maps a city to its id

In [99]:
cur.execute("SELECT * FROM city;")

rows = cur.fetchall()
city_to_id = {}
for row in rows:
  city_to_id[(row[1], row[2])] = row[0]

---
#### Percent_over_25_completed_highschool

Insert Data Into Percent_over_25_completed_highschool Table

In [100]:
# Given city and state, return city id
def convert_state_and_city_to_id(city, state):
  if (city, state_to_id[state]) in city_to_id:
    return city_to_id[(city, state_to_id[state])]
  return None

In [101]:
# Rename columns to match table colum names
this_completed_highschool = completed_highschool_df.rename(columns={'City': 'city_id', "Geographic Area": "state", "percent_completed_hs": "percentage"})

# Convert cities to city ids
this_completed_highschool["city_id"] = this_completed_highschool.apply(lambda x: convert_state_and_city_to_id(x["city_id"], x["state"]), axis=1)

# Drop state columns
this_completed_highschool.drop("state", inplace=True, axis=1)

# Remove colums with percentage "-"
this_completed_highschool = this_completed_highschool[this_completed_highschool["percentage"] != "-"]

this_completed_highschool.to_sql("percent_over_25_completed_highschool", conn, if_exists='append', index = False)

29132

---
#### Race

Insert data into Race Table

In [102]:
races = [
    ("white",),
    ("black",),
    ("native_american",),
    ("asian",),
    ("hispanic",),
    ("other",)
]

insert_race_query = "INSERT INTO race (name) VALUES(?);"

cur.executemany(insert_race_query, races)
conn.commit()

Create Dict that maps a race to its id

In [103]:
cur.execute("SELECT * FROM race;")

rows = cur.fetchall()
race_to_id = {}
for row in rows:
  race_to_id[row[1]] = row[0]

---
#### Share_race_by_city

Insert data into Share_race_by_city Table

In [104]:
# Remove city, town or CDP from city name (so that police killings cities match)
remove_redundency(race_by_city_df)
race_by_city_df = race_by_city_df[(race_by_city_df['share_white'] != '(X)') | (race_by_city_df['share_black'] != '(X)') | (race_by_city_df['share_native_american'] != '(X)') | (race_by_city_df['share_asian'] != '(X)') | (race_by_city_df['share_hispanic'] != '(X)')]

race_by_city_list = []

# For each row which has shares for all 5 races, create 5 rows corresponding to each specific race
for index, row in race_by_city_df.iterrows():
  city_id = convert_state_and_city_to_id(row[1], row[0])

  if (city_id == None):
    continue
  
  white_share = {
    "city_id": city_id, 
    "race_id": race_to_id["white"],
    "share": row[2]
  }
  black_share = {
    "city_id": city_id, 
    "race_id": race_to_id["black"],
    "share": row[3]
  }
  native_american_share = {
    "city_id": city_id, 
    "race_id": race_to_id["native_american"],
    "share": row[4]
  }
  asian_share = {
    "city_id": city_id, 
    "race_id": race_to_id["asian"],
    "share": row[5]
  }
  hispanic_share = {
    "city_id": city_id, 
    "race_id": race_to_id["hispanic"],
    "share": row[6]
  }
  race_by_city_list += [white_share, black_share, native_american_share, asian_share, hispanic_share]

race_by_city_df = pd.DataFrame(race_by_city_list)

race_by_city_df.to_sql("share_race_by_city", conn, if_exists='append', index = False)

145665

---
#### Police_killings

Insert data into Police_killings Table

In [105]:
# Given race acronym, get race id
def convert_string_to_race_id(race_string):
    if not isinstance(race_string, str):
        return None

    char_to_race = {
        "W": "white",
        "B": "black",
        "N": "native_american",
        "A": "asian",
        "H": "hispanic",
        "O": "other"
    }
    
    race = char_to_race[race_string]

    return race_to_id[race]

# Given date string, convert fo format adequate for SQLite
def correct_date_format(date_string):
    date_split = date_string.split("/")
    return f"{date_split[2]}-{date_split[1]}-{date_split[0]}"

In [106]:
# Convert city to city id (if exists)
police_killings_df["city"] = police_killings_df.apply(lambda x: convert_state_and_city_to_id(x["city"], x["state"]), axis=1)

# Convert race to race id (if exists)
police_killings_df["race"] = police_killings_df["race"].map(convert_string_to_race_id)

# Convert race to race id (if exists)
police_killings_df["date"] = police_killings_df["date"].apply(correct_date_format)

# Drop state columns
police_killings_df.drop("state", inplace=True, axis=1)

# Rename columns to match table colum names
police_killings_df = police_killings_df.rename(columns={'city': 'city_id', "state": "state_id", "race": "race_id"})

# Drop column id (id's will be autoincremental)
police_killings_df.drop("id", inplace=True, axis=1)

police_killings_df.to_sql("police_killings", conn, if_exists='append', index = False)

2535

### Close Connection

In [107]:
conn.close()